# Overview

Before building the base GRN, we need to annotate the coaccessible peaks and filter our active promoter/enhancer elements. First, we will identify the peaks around transcription starting sites (TSS). We will then merge the Cicero data with the TSS peak information and filter any peaks with weak connections to the TSS peaks. As such, the filtered peak data will only include TSS peaks and peaks with strong TSS connections. These will be our active promoter/enhancer elements for our base GRN. 

### Notebook file

Notebook file is available on CellOracle GitHub page.
https://github.com/morris-lab/CellOracle/blob/master/docs/notebooks/01_ATAC-seq_data_processing/option1_scATAC-seq_data_analysis_with_cicero/02_preprocess_peak_data.ipynb



# 0. Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns


import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm


In [2]:
from celloracle import motif_analysis as ma
import celloracle as co
co.__version__

/home/aregano/anaconda3/envs/celloracle_env/lib/python3.8/site-packages/loompy/bus_file.py:68: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def twobit_to_dna(twobit: int, size: int) -> str:
/home/aregano/anaconda3/envs/celloracle_env/lib/python3.8/site-packages/loompy/bus_file.py:85: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def dna_to_twobit(dna: str) -> i

'0.14.0'

In [3]:
%config InlineBackend.figure_format = 'retina'

plt.rcParams['figure.figsize'] = [6, 4.5]
plt.rcParams["savefig.dpi"] = 300

# 1. Load scATAC peak data and peak connection data made with Cicero

## 1.0. Download data

In this notebook, we will annotate and filter output from Cicero. Please refer to the previous step to learn about data preparation with Cicero.
https://morris-lab.github.io/CellOracle.documentation/tutorials/base_grn.html#step1-scatac-seq-analysis-with-cicero


Here, we will use the preprocessed fetal brain scATAC-seq data from step 1.


You can download the demo file by running the following command. 

Note: If the download fails, please manually download and unzip the data.
https://raw.githubusercontent.com/morris-lab/CellOracle/master/docs/demo_data/all_peaks.csv

https://raw.githubusercontent.com/morris-lab/CellOracle/master/docs/demo_data/cicero_connections.csv

In [4]:
# Download file. 
!wget https://raw.githubusercontent.com/morris-lab/CellOracle/master/docs/demo_data/all_peaks.csv
!wget https://raw.githubusercontent.com/morris-lab/CellOracle/master/docs/demo_data/cicero_connections.csv
    
# If you are using macOS, please try the following command.
#!curl -O https://raw.githubusercontent.com/morris-lab/CellOracle/master/docs/demo_data/all_peaks.csv
#!curl -O https://raw.githubusercontent.com/morris-lab/CellOracle/master/docs/demo_data/cicero_connections.csv


--2024-02-15 11:03:10--  https://raw.githubusercontent.com/morris-lab/CellOracle/master/docs/demo_data/all_peaks.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2940392 (2.8M) [text/plain]
Saving to: ‘all_peaks.csv’

all_peaks.csv       100%[===================>]   2.80M  1.98MB/s    in 1.4s    

2024-02-15 11:03:11 (1.98 MB/s) - ‘all_peaks.csv’ saved [2940392/2940392]

--2024-02-15 11:03:12--  https://raw.githubusercontent.com/morris-lab/CellOracle/master/docs/demo_data/cicero_connections.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 

## 1.1. Load data

In [5]:
# Load scATAC-seq peak list.
peaks = pd.read_csv("all_peaks.csv", index_col=0)
peaks = peaks.x.values
peaks

array(['chr10_100006139_100006389', 'chr10_100015291_100017830',
       'chr10_100018677_100020384', ..., 'chrY_90804622_90805450',
       'chrY_90808626_90809117', 'chrY_90810560_90811167'], dtype=object)

### Zhu et al ATAC data

In [12]:
os.getcwd()

'/run/user/1001/gvfs/smb-share:server=tierra,share=sc/LAB_RB/LAB/Alvaro/Bioinformatics/Analysis/scAGM_Embryos/Notebooks/CellOracle/scATAC_data'

In [15]:
load_dir = "../../../celloracle/ATAC_Zhu/"

In [17]:
"".join([load_dir, "all_peaks.csv"])

'../../../celloracle/ATAC_Zhu/all_peaks.csv'

In [20]:
peaks = pd.read_csv("".join([load_dir, "all_peaks_Zhu_et_al.csv"]), index_col=0)
# peaks = pd.read_csv("all_peaks.csv", index_col=0)

peaks = peaks.x.values
peaks

array(['chr1_10007346_10007524', 'chr1_10008736_10009547',
       'chr1_100180128_100180379', ..., 'chrY_90808741_90808915',
       'chrY_90810639_90811031', 'chrY_90812853_90813226'], dtype=object)

In [21]:
# Load Cicero coaccessibility scores.
cicero_connections = pd.read_csv("".join([load_dir, "cicero_connections_Zhu_et_al.csv"]), index_col=0)
# cicero_connections = pd.read_csv("cicero_connections.csv", index_col=0)
cicero_connections.head()

,Peak1,Peak2,coaccess
1,chr10_100013963_100014429,chr10_99750232_99750406,0.001335
2,chr10_100013963_100014429,chr10_99759642_99759789,0.000000
3,chr10_100013963_100014429,chr10_99829469_99829829,0.000000
4,chr10_100013963_100014429,chr10_99834942_99835260,0.000000
5,chr10_100013963_100014429,chr10_99839536_99839644,0.019235


# 2. Annotate transcription start sites (TSSs)¶
## IMPORTANT: Please make sure that you are setting correct reference genoms.
 If your scATAC-seq data was generated with mm10 reference genome, please set `ref_genome="mm10"`.
 
You can check supported reference genome using `ma.SUPPORTED_REF_GENOME`

 If your reference genome is not in the list, please send a request to us through CellOracle GitHub issue page.

In [22]:
ma.SUPPORTED_REF_GENOME

,species,ref_genome,provider
0,Human,hg38,UCSC
1,Human,hg19,UCSC
2,Mouse,mm39,UCSC
3,Mouse,mm10,UCSC
4,Mouse,mm9,UCSC
5,S.cerevisiae,sacCer2,UCSC
6,S.cerevisiae,sacCer3,UCSC
7,Zebrafish,danRer7,UCSC
8,Zebrafish,danRer10,UCSC
9,Zebrafish,danRer11,UCSC


In [23]:
##!! Please make sure to specify the correct reference genome here
tss_annotated = ma.get_tss_info(peak_str_list=peaks, ref_genome="mm10") 

# Check results
tss_annotated.tail()

que bed peaks: 150427
tss peaks in que: 19169


,chr,start,end,gene_short_name,strand
19164,chr6,94499812,94500702,Slc25a26,+
19165,chr19,45659338,45660659,Fbxw4,-
19166,chr12,100899000,100899625,Gpr68,-
19167,chr4,129490910,129491359,Fam229a,-
19168,chr5,21035793,21036035,Ptpn12,-


# 3. Integrate TSS info and cicero connections

The output file after the integration process has three columns: `["peak_id", "gene_short_name", "coaccess"`].

- "peak_id" is either the TSS peak or the peaks that have a connection to a TSS peak.
- "gene_short_name" is the gene name that associated with the TSS site. 
- "coaccess" is the coaccessibility score between the peak and a TSS peak. If the score is 1, it means that the peak is a TSS itself.

In [24]:
integrated = ma.integrate_tss_peak_with_cicero(tss_peak=tss_annotated, 
                                               cicero_connections=cicero_connections)
print(integrated.shape)
integrated.head()

(438635, 3)


,peak_id,gene_short_name,coaccess
0,chr10_100015425_100016651,Kitl,1.000000
1,chr10_100015425_100016651,Tmtc3,0.306834
2,chr10_100019045_100019149,Kitl,0.138011
3,chr10_100019045_100019149,Tmtc3,0.056436
4,chr10_100019989_100020394,Kitl,0.069377


# 4. Filter peaks
Remove peaks with weak coaccessibility scores.

In [25]:
peak = integrated[integrated.coaccess >= 0.8]
peak = peak[["peak_id", "gene_short_name"]].reset_index(drop=True)

In [26]:
print(peak.shape)
peak.head()

(18056, 2)


,peak_id,gene_short_name
0,chr10_100015425_100016651,Kitl
1,chr10_100486568_100487889,Tmtc3
2,chr10_100588506_100589498,4930430F08Rik
3,chr10_100741132_100741585,Gm35722
4,chr10_100741989_100742521,Gm35722


# 5. Save data
Save the promoter/enhancer peaks.

In [27]:
peak.to_csv("".join([load_dir, "processed_peak_file_Zhu_et_al.csv"]))

# peak.to_csv("processed_peak_file.csv")

**Please go to next step: Transcriptoin factor motif scan**

https://morris-lab.github.io/CellOracle.documentation/tutorials/motifscan.html